In [1]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
#https://ddolcat.tistory.com/674  참조 , 셀레니엄 버전 문제해결
import time
from tqdm.notebook import tqdm
from selenium.webdriver.common.by import By

driver.get('http://www.melon.com')
time.sleep(2)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.68M/6.68M [00:01<00:00, 6.97MB/s]
C:\Users\98083\anaconda3\envs\suwah\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """


In [20]:
#멜론 차트 클릭 후 2초 정지
driver.find_element(By.CSS_SELECTOR, ".menu_bg.menu01").click()
time.sleep(2)

In [21]:
#일간차트 클릭
driver.find_element(By.CSS_SELECTOR, ".menu_chart.m2").click()

In [22]:
import re

In [23]:
title= driver.find_elements(By.CLASS_NAME, "ellipsis.rank01")

title2=[]
for i in title:
    title2.append(i.text)

del title2[101:]

In [24]:
singer=driver.find_elements(By.CLASS_NAME, "ellipsis.rank02")
singer2=[]
for i in singer:
    singer2.append(i.text)
    
del singer2[101:]

In [25]:
songTagList = driver.find_elements(By.XPATH, '/html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr')

number=[]
for i in songTagList:
    number.append(i.get_attribute('data-song-no'))

해결해야 됐던 문제
1. 윗 부분에서 LYRIC 크롤링이 또 되지 않음
-> 수정된 html문으로 재정비


In [26]:
LYRIC=[]
LYRIC2=[]

for i in number:
    driver.get("https://www.melon.com/song/detail.htm?songId="+ i)
    lyric=driver.find_element(By.CLASS_NAME, "lyric")
    LYRIC.append(lyric.text)
    

In [19]:
LYRIC

[]

In [18]:
LYRIC=[]
for i in LYRIC:
    LYRIC2.append(i.replace("\n",""))
    
  # 줄 개행 삭제 필요시 사용, 근데 토큰화 과정에서 줄 개행 문자 생략됨
  

In [17]:
df1 = pd.DataFrame(data=title2, columns=["제목"])
df2 = pd.DataFrame(data=singer2, columns=["가수"])
df3 = pd.DataFrame(data=LYRIC, columns=["가사"])


result1 = pd.concat([df1, df2, df3], axis=1)
result1.to_csv("멜론top100크롤링완성.csv", encoding='utf-8')


ValueError: All arrays must be of the same length
가 났습니다..하하하하

df병합(merge를 사용하면 안되서 concat을 사용함, axis는 좌우(열방향)으로 이어붙이기 위함)
https://yganalyst.github.io/data_handling/Pd_12/ 참조

ValueError: No engine for filetype: 'csv'
기존에 했던 건 엑셀이라 깨짐 현상 있어 csv버전으로 재시도, to_excel->to_csv

In [31]:
df = pd.read_csv('멜론top100크롤링완성.csv', sep = ",", engine='python', encoding="utf-8")
df.dropna(axis=0)
df.to_csv('melontop100.tsv', sep='\t', encoding ="utf-8", index=False)